In [1]:
from datetime import datetime, timedelta
from typing import List

from pydantic import BaseModel, Field

class Orbit(BaseModel):
    name = str = Field(..., description="Name")
    tle = List[str] = Field(..., description="Orbit")

class PropagationRequest(BaseModel):
    orbit: Orbit = Field(
        ...,
        description="Orbit to be propagated."
    )
    start: datetime = Field(
        ...,
        description="Propagation start time."
    )
    duration: timedelta = Field(
        ...,
        description="Propagation duration."
    )
    time_step: timedelta = Field(
        ...,
        description="Propagation time step duration."
    )

class PropagationRecord(BaseModel):
    time: datetime = Field(..., description="Time")
    position: List[float] = Field(..., description="Position")
    velocity: List[float] = Field(..., description="Velocity")

class PropagationResponse(BaseModel):
    records: List[PropagationRecord] = Field(
        [],
        description="Propagation results"
    )

In [ ]:
from tatc.schemas import Satellite, TwoLineElements
from tatc.analysis import collect_orbit_track

import pandas as pd

def propagate(request: PropagationRequest) -> PropagationResponse:
    return PropagationResponse(
        records=collect_orbit_track(
            Satellite(
                name=request.orbit.name,
                orbit=TwoLineElements(
                    request.orbit.tle[0],
                    request.orbit.tle[1]
                )
            ),
            None,
            pd.date_range(request.start, request.start + request.duration, freq=request.time_step)
        ).apply(lambda r: PropagationRecord(
            time=r.time,
            position=r.geometry,
            velocity=r.velocity
        ), axis=1)
    )